In [55]:
import os
from dotenv import load_dotenv
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import GoogleDriveLoader
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone

load_dotenv(override=True)

embeddings = OpenAIEmbeddings()

index_name = os.getenv("PINECONE_INDEX_NAME")

In [57]:
loader = GoogleDriveLoader(document_ids=["1YfimQqX74jEjHCZPIXWHy8hv6TZ9Yh_k4EhMs27nCr0"],
                          credentials_path="credentials.json",
                          load_extended_matadata=True,
                          token_path=".credentials/token.json",
                          supportsAllDrives=True
                          )
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
documents = text_splitter.split_documents(docs)
print(documents[0:3])

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index(index_name)
index.delete(
    filter={
        "source": {"$eq": "documentary"},
    }
)

pinecone = PineconeVectorStore.from_documents(
    documents, embeddings, index_name=index_name
)


[Document(metadata={'source': 'https://docs.google.com/document/d/1YfimQqX74jEjHCZPIXWHy8hv6TZ9Yh_k4EhMs27nCr0/edit', 'title': 'BUSINESS DOC', 'when': '2024-08-13T06:02:46.375Z'}, page_content='\ufeffSTUFF ON BUSINESS')]


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 13 Aug 2024 07:30:42 GMT', 'Content-Type': 'application/json', 'Content-Length': '115', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '37', 'x-pinecone-request-id': '4991704815229236971', 'x-envoy-upstream-service-time': '38', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Serverless and Starter indexes do not support deleting with metadata filtering.","details":[]}
